In [206]:
from BeautifulSoup import BeautifulSoup as BS
#import pandas as pd
from StringIO import StringIO
from collections import namedtuple
import os
from pprint import pprint
from HTMLParser import HTMLParser

In [207]:
html=open("/Users/pg/code/hamsa/listing.html").read()

In [217]:
def parser_product(html):
    soup=BS(html)

    Entry=namedtuple('Entry', ['product_id','product', 'price_usd', "price_btc",
                     "vendor","vendor_url","product_type",
                    'delivery'])

    li = soup.findAll("div",{"class":"row"})[1].find("div",{'class':"col-md-8"})


    entry = { 'product_id':li.form.get('action'),
    'product':li.h2.text,
    'price_usd':li.find("div",{"class":"listing-price"}).strong.text,
    'price_btc':li.find("div",{"class":"listing-price"}).span.text,
    'vendor':li.tr.findAll("td")[1].a.text,
    'vendor_url':li.tr.findAll("td")[1].a["href"], 
    'product_type':li.findAll("td")[3].text,        
    'delivery':li.findAll("td")[5].text,        
    #'description':soup.p.text
    }

    return Entry(**entry)

In [277]:
def parser_feedback(html):
    soup=BS(html)
    feedbacks=[]
    li = soup.findAll("div",{"class":"row"})[1].find("div",{'class':"col-md-8"})
    prod=li.form.get('action')
    Feedback = namedtuple("Feedback", ["date",'user',"delivery_time", "note", "text"])
    h=HTMLParser()
    try:
        for e in soup.find("table",{"class":"table"}).tbody.findAll("tr"):
            feed,text,deliv,user,date=e.findAll('td')
            feed=feed.span.get("class").split()[1].split("-")[1] 
            note=0
            if feed == 'danger':
                note=-1
            if feed == "default":
                note=0
            if feed == "success":
                note=1
        #print "||".join([date.text,user.text, h.unescape(text.text),  deliv.text])
            f=Feedback(date.text,user.text,deliv.text, note, text.text)
            feedbacks.append(f)
    except AttributeError:
        pass
    return feedbacks
        

In [266]:
def parser_vendor(html):
    
    Vendor = namedtuple("Vendor", ["vendor_url", 'name', 
                                   'last_seen', 'since', 'order_count', 
                                  'avg_volume', "pos_fb", 'neutral_fb', 'neg_fb'])
    soup=BS(html)
    x=soup.find("div", {"class":"col-md-12", "style":"background-color: #f5f5f5;"})
    y=nn.find("div",  {'class':"row row-vendor-profile"})
    
    vendor_url = x.h1.a.get("href")
    vendor = x.h1.contents[0]
    
    last_seen=x.find("p", {"class":"text-muted"}).contents[0].split()[3]
    since = x.find("p", {"class":"text-muted"}).contents[2].split()[4]
    
    order_count = y.findAll("div", {'class':"col-md-3 text-center"})[0].p.strong.text
    avg_volume=y.findAll("div", {'class':"col-md-3 text-center"})[1].p.strong.text
    
    pos_fb = y.findAll("div", {'class':"col-md-4"})[0].p.strong.text
    neutral_fb = y.findAll("div", {'class':"col-md-4"})[1].p.strong.text
    neg_fb = y.findAll("div", {'class':"col-md-4"})[2].p.strong.text
    
    return Vendor(vendor_url,vendor,last_seen, since,order_count, avg_volume, pos_fb, neutral_fb,neg_fb)
    
#parser_product(d)

In [271]:
def fetch_tor(url):
    #url='http://hansamkt2rr6nfg3.onion/vendor/ProfessorDark/'
    cmd="curl --socks4a 127.0.0.1:9150 "+ url +" -A 'Mozilla/5.0 (Windows NT 6.1; rv:45.0) Gecko/20100101 Firefox/45.0' "
    return os.popen(cmd).read()


In [272]:
#vendor
url='http://hansamkt2rr6nfg3.onion/vendor/ProfessorDark/'
d=fetch_tor(url)
parser_vendor(d)
#parser_feedback(d)
#nn.form.get('action')

Vendor(vendor_url=u'/store/ProfessorDark/', name=u'ProfessorDark ', last_seen=u'2016-12-06', since=u'2015-11-08', order_count=u'1000+', avg_volume=u'0.00549751', pos_fb=u'2929', neutral_fb=u'108', neg_fb=u'38')

In [273]:
#product
url='http://hansamkt2rr6nfg3.onion/listing/39972/feedback/'
d=fetch_tor(url)
parser_product(d)

Entry(product_id=u'/listing/39972/', product=u'5g Sour Diesel Bio A+++ (Medical Grade) !!Shipping From Germany!!', price_usd=u'USD 53.71', price_btc=u'0.0703', vendor=u'GermanDrugsShippers', vendor_url=u'/vendor/GermanDrugsShippers/', product_type=u'Physical', delivery=u'Germany')

In [278]:
parser_feedback(d)

[Feedback(date=u'2016-12-03 15:51 UTC', user=u's***8', delivery_time=u'Next Day', note=1, text=u'Schnelle lieferung. Weed ist ok'),
 Feedback(date=u'2016-12-02 13:47 UTC', user=u'A***e', delivery_time=u'3 days', note=1, text=u'very fast, very good quality ;-))'),
 Feedback(date=u'2016-11-21 18:58 UTC', user=u'M***t', delivery_time=u'Not specified', note=1, text=u'Stealth was bad. The product itself is fantastic though, I got one big nug and some shake but the quality is really good.'),
 Feedback(date=u'2016-11-18 08:20 UTC', user=u'M***s', delivery_time=u'5 days', note=1, text=u'Goods are ok, great smell, great smoke, stealth is simple yet effective, would recommend this seller. A+'),
 Feedback(date=u'2016-11-07 14:08 UTC', user=u'J***d', delivery_time=u'3 days', note=1, text=u'Awesome like everytime!'),
 Feedback(date=u'2016-11-03 14:49 UTC', user=u'p***x', delivery_time=u'2 days', note=1, text=u'Not as good as I expected.Some &quot;extra sticks&quot;from cutoffs inside...Overall it&#